In [16]:
import torch

In [3]:
with open("shakespeare.txt", encoding='utf-8') as f:
    text=f.read()

In [5]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [6]:
len(text)

1115394

In [9]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(chars)
print(''.join(chars))
print(len(chars))

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [13]:
stoi={s:i for i,s in enumerate(chars)}
itos={i:s for i,s in enumerate(chars)}
encode=lambda s: [stoi[c] for c in s]
decode=lambda i: ''.join([itos[d] for d in i])

In [15]:
print(encode('hii there!'))
print(decode(encode('hii there!')))

[46, 47, 47, 1, 58, 46, 43, 56, 43, 2]
hii there!


In [17]:
data=torch.tensor(encode(text),dtype=torch.long)

In [19]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [22]:
type(data), data.dtype, data.shape

(torch.Tensor, torch.int64, torch.Size([1115394]))

In [28]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [29]:
print(train_data.shape)

torch.Size([1003854])


In [39]:
torch.manual_seed=1337
batch_size=4
block_size=8

def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size, (batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    print(x.shape)
    print(x)
    print(ix)
    
get_batch('train')

torch.Size([4, 8])
tensor([[53, 52, 43,  1, 57, 53,  1, 58],
        [43,  1, 61, 43, 56, 43,  1, 42],
        [43, 57, 57,  1, 44, 43, 39, 56],
        [61, 47, 58, 46,  1, 58, 46, 43]])
tensor([400173, 174132,  74063, 776483])
